In [3]:
import requests
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.proxy import Proxy, ProxyType
import random 

In [4]:
text = 'なぜ日本人は漢字を使うのか'
page_url = "https://google.com/search?q="+text
driver_path = r'C:\Users\Clsm\Documents\GitHub\test_files\chromedriver.exe'

In [5]:
driver = webdriver.Chrome(driver_path)
driver.get(page_url)
time.sleep(5)
htmlContent = driver.page_source
print(len(htmlContent))

6333


In [6]:
soup = BeautifulSoup(htmlContent, 'html.parser')
body = soup.find('body')
body

<body onload="e=document.getElementById('captcha');if(e){e.focus();} if(solveSimpleChallenge) {solveSimpleChallenge(,);}" style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px;">
<div style="max-width:400px;">
<hr noshade="" size="1" style="color:#ccc; background-color:#ccc;"/><br/>
<form action="index" id="captcha-form" method="post">
<noscript>
<div style="font-size:13px;">
  In order to continue, please enable javascript on your web browser.
</div>
</noscript>
<script async="" crossorigin="anonymous" integrity="sha384-jffSm4FBmQyLvL1V8BXFUBdZCFkPLi8N+X9NGYs2YKU4uUiYzy53t/3mlwj1fdwI" src="https://www.gstatic.com/recaptcha/releases/vpEprwpCoBMgy-fvZET0Mz6L/recaptcha__en.js" type="text/javascript"></script><script async="" defer="" src="https://www.google.com/recaptcha/api.js"></script>
<script>var submitCallback = function(response) {document.getElementById('captcha-form').submit();};</script>
<div class="g-recaptcha" data-callback="s

In [7]:
from html.parser import HTMLParser
from urllib import parse
class LinkFinder(HTMLParser):
    def __init__(self, page_url) :
        super().__init__()
        self.page_url = page_url
        self.links = list()
    
    def handle_starttag(self, attrs):
        for attr in attrs:
            for (attribute, value) in attr.items():
                if 'https://' in value:
                    if 'https://webcache.googleusercontent.com' in value:
                        self.links.append(value)
                
                    elif 'google' not in value:
                        self.links.append(value)  
    def all_links(self, attrs):              
        for attr in attrs:
            for (attribute, value) in attr.items():
                if 'https://' in value:
                    self.links.append(value)

    def error(self, message: str) -> str:
        return super().error(message)
    def find_pageurl(self):
        return self.links

In [8]:
tag = [tag for tag in soup.findAll()]
attrs = list()
for t in tag:
    attrs.append(t.attrs)
print(attrs)

[{}, {}, {'http-equiv': 'content-type', 'content': 'text/html; charset=utf-8'}, {'name': 'viewport', 'content': 'initial-scale=1'}, {}, {'style': 'font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px;', 'onload': "e=document.getElementById('captcha');if(e){e.focus();} if(solveSimpleChallenge) {solveSimpleChallenge(,);}"}, {'style': 'max-width:400px;'}, {'noshade': '', 'size': '1', 'style': 'color:#ccc; background-color:#ccc;'}, {}, {'id': 'captcha-form', 'action': 'index', 'method': 'post'}, {}, {'style': 'font-size:13px;'}, {'type': 'text/javascript', 'async': '', 'src': 'https://www.gstatic.com/recaptcha/releases/vpEprwpCoBMgy-fvZET0Mz6L/recaptcha__en.js', 'crossorigin': 'anonymous', 'integrity': 'sha384-jffSm4FBmQyLvL1V8BXFUBdZCFkPLi8N+X9NGYs2YKU4uUiYzy53t/3mlwj1fdwI'}, {'src': 'https://www.google.com/recaptcha/api.js', 'async': '', 'defer': ''}, {}, {'id': 'recaptcha', 'class': ['g-recaptcha'], 'data-sitekey': '6LfwuyUTAAAAAOAmoS0fdqijC2

In [9]:
finder = LinkFinder(page_url=page_url)
finder.handle_starttag(attrs=attrs)

In [10]:
print(finder.find_pageurl())

['https://www.gstatic.com/recaptcha/releases/vpEprwpCoBMgy-fvZET0Mz6L/recaptcha__en.js']


In [11]:
finder.all_links(attrs = attrs)
all_links_list = finder.find_pageurl()

In [18]:
import tldextract
import re
def check_cache(all_link_list):
    actual_from_cache = []
    webcache_list = []
    cache_name = 'www'
    
    # check url http:// or https:// or www. or domain
    url_valid_regex = r'(?:(?:https?|ftp):\/\/)?[\w]+\.[\w/\-&?=%.]+'
    actual_regex = r'(http[s]?.*|www.*){2}'
    
    # check url starting http://domain or https://domain
#     url_valid_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+' 
    
    for url in all_link_list:
        
        url_extractor = tldextract.extract(url)
        is_url_valid = re.findall(url_valid_regex, url)
        
        if is_url_valid and url_extractor.subdomain == cache_name:
            webcache_list.append(url)
            is_actual_from_cache = re.findall(actual_regex, url)
            if is_actual_from_cache:
                actual_from_cache.append(is_actual_from_cache[0].split('+')[0])
    
    print(f'Total webcache links: {len(webcache_list)}')
    print('Total links: ', len(all_link_list))
    return webcache_list, actual_from_cache

In [19]:
print(check_cache(all_links_list))

Total webcache links: 5
Total links:  6
(['https://www.gstatic.com/recaptcha/releases/vpEprwpCoBMgy-fvZET0Mz6L/recaptcha__en.js', 'https://www.gstatic.com/recaptcha/releases/vpEprwpCoBMgy-fvZET0Mz6L/recaptcha__en.js', 'https://www.google.com/recaptcha/api.js', 'https://www.google.com/recaptcha/api2/anchor?ar=1&k=6LfwuyUTAAAAAOAmoS0fdqijC2PbbdH4kjq62Y1b&co=aHR0cHM6Ly93d3cuZ29vZ2xlLmNvbTo0NDM.&hl=en&v=vpEprwpCoBMgy-fvZET0Mz6L&size=normal&s=LZKyIOqMLMPx6b2jroV4A-kBm0vLSI92Gi8sj53qy3z-t2RGVKBYLLGkYOr_NRIjVXaGljrTkI6kA2JBj2Ujyg_eMV8mNdT556dCXTEcmDEgLlZ0dug_mK5MbD8Sy_-VZYENBunBO0btNsqZrzdXugvXFjD6fVmTFb-dRA90NaqRHmn1DAqoJyuuWIPRhZOcPJ8nBu4MQvU82OcuL56ja1neCwu1GthNkV32sDUEhjc8SsTXZ9NLeR2IcrZerbdP3xxCx42zvYrxnM2NnXtWzYochwwLUyI&cb=jpaw75fww1m0', 'https://www.google.com/recaptcha/api2/bframe?hl=en&v=vpEprwpCoBMgy-fvZET0Mz6L&k=6LfwuyUTAAAAAOAmoS0fdqijC2PbbdH4kjq62Y1b'], ['www.gstatic.com/recaptcha/releases/vpEprwpCoBMgy-fvZET0Mz6L/recaptcha__en.js', 'www.gstatic.com/recaptcha/releases/vpEprwpCoB